# Fredo

In [ ]:
import pandas as pd
df = pd.read_csv('./Dataset/ML_cars.csv')
df.head()

In [ ]:
df.isna().sum()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style= 'whitegrid' , context='notebook')
cols = ['horsepower','carwidth','enginesize','curbweight','price']
sns.pairplot(df[cols], height = 2.5)
plt.show()

In [ ]:
sns.set(font_scale = 1.5)
sns.heatmap(df[cols].corr(), cbar = True, annot=True, yticklabels=cols, xticklabels=cols);